In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.ml.feature import StringIndexer
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [3]:
sc = SparkContext(appName = 'Lab5_pt2')

In [4]:
ss = SparkSession.builder.master('local').getOrCreate()

In [8]:
dados_time_futebol = sc.textFile('dataset2.csv')

In [9]:
dados_time_futebol.take(5)

['media_faltas_sofridas,media_faltas_recebidas,media_cartoes_recebidos,media_chutes_a_gol,resultado',
 '4.8,3,1.4,0.3,vitoria',
 '5.1,3.8,1.6,0.2,vitoria',
 '4.6,3.2,1.4,0.2,vitoria',
 '5.3,3.7,1.5,0.2,vitoria']

In [15]:
dados_time_futebol_2 = dados_time.filter(lambda x:'media_faltas_sofridas' not in x)

### Limpeza dos dados 

In [16]:
dados_time_futebol_3 = dados_time_futebol_2.map(lambda x:x.split(','))

In [17]:
#Mapeando as colunas 
dados_time_futebol_4 = dados_time_futebol_3.map(lambda p: Row(media_faltas_sofridas = float(p[0]), 
                                                              media_faltas_recebidas = float(p[1]), 
                                                              media_cartoes_recebidos = float(p[2]), 
                                                              media_chutes_a_gol = float(p[3]), 
                                                              resultado = p[4] ))

In [18]:
df_time = ss.createDataFrame(dados_time_futebol_4)
df_time.cache()

DataFrame[media_faltas_sofridas: double, media_faltas_recebidas: double, media_cartoes_recebidos: double, media_chutes_a_gol: double, resultado: string]

In [19]:
df_time.take(5)

[Row(media_faltas_sofridas=4.8, media_faltas_recebidas=3.0, media_cartoes_recebidos=1.4, media_chutes_a_gol=0.3, resultado='vitoria'),
 Row(media_faltas_sofridas=5.1, media_faltas_recebidas=3.8, media_cartoes_recebidos=1.6, media_chutes_a_gol=0.2, resultado='vitoria'),
 Row(media_faltas_sofridas=4.6, media_faltas_recebidas=3.2, media_cartoes_recebidos=1.4, media_chutes_a_gol=0.2, resultado='vitoria'),
 Row(media_faltas_sofridas=5.3, media_faltas_recebidas=3.7, media_cartoes_recebidos=1.5, media_chutes_a_gol=0.2, resultado='vitoria'),
 Row(media_faltas_sofridas=5.1, media_faltas_recebidas=3.5, media_cartoes_recebidos=1.4, media_chutes_a_gol=0.2, resultado='vitoria')]

In [22]:
#Criando um index numerico para a coluna de label target
stringIndexer = StringIndexer(inputCol = 'resultado',outputCol = 'idx_resultado')

In [24]:
si_model = stringIndexer.fit(df_time)

In [26]:
df_time_final = si_model.transform(df_time)

In [31]:
df_time_final.select('resultado','idx_resultado').distinct().collect()

[Row(resultado='derrota', idx_resultado=0.0),
 Row(resultado='vitoria', idx_resultado=2.0),
 Row(resultado='empate', idx_resultado=1.0)]

In [32]:
df_time_final.show()

+---------------------+----------------------+-----------------------+------------------+---------+-------------+
|media_faltas_sofridas|media_faltas_recebidas|media_cartoes_recebidos|media_chutes_a_gol|resultado|idx_resultado|
+---------------------+----------------------+-----------------------+------------------+---------+-------------+
|                  4.8|                   3.0|                    1.4|               0.3|  vitoria|          2.0|
|                  5.1|                   3.8|                    1.6|               0.2|  vitoria|          2.0|
|                  4.6|                   3.2|                    1.4|               0.2|  vitoria|          2.0|
|                  5.3|                   3.7|                    1.5|               0.2|  vitoria|          2.0|
|                  5.1|                   3.5|                    1.4|               0.2|  vitoria|          2.0|
|                  4.9|                   3.0|                    1.4|               0.2

In [33]:
#Correlação entre variáveis 
for col in df_time_final.columns:
    if not(isinstance(df_time_final.select(col).take(1)[0][0],str)):
        print('Correlação entre ',col,'index_resultado: ',df_time_final.stat.corr('idx_resultado',col))

Correlação entre  media_faltas_sofridas index_resultado:  -0.4600391565002369
Correlação entre  media_faltas_recebidas index_resultado:  0.6183715308237437
Correlação entre  media_cartoes_recebidos index_resultado:  -0.649241830764174
Correlação entre  media_chutes_a_gol index_resultado:  -0.5803770334306265
Correlação entre  idx_resultado index_resultado:  1.0


In [63]:
#Escolhendo as features de maior correlação com a variável alvo
def transformaRow (row):
    obj = (row['resultado'],row['idx_resultado'],Vectors.dense([row['media_faltas_sofridas'],
                                                            row['media_faltas_recebidas'],
                                                              row['media_cartoes_recebidos'],
                                                              row['media_chutes_a_gol']]))
    return obj

In [65]:
df_time_final_rdd = df_time_final.rdd.map(transformaRow)

In [70]:
df_time_final_rdd.take(5)

[('vitoria', 2.0, DenseVector([4.8, 3.0, 1.4, 0.3])),
 ('vitoria', 2.0, DenseVector([5.1, 3.8, 1.6, 0.2])),
 ('vitoria', 2.0, DenseVector([4.6, 3.2, 1.4, 0.2])),
 ('vitoria', 2.0, DenseVector([5.3, 3.7, 1.5, 0.2])),
 ('vitoria', 2.0, DenseVector([5.1, 3.5, 1.4, 0.2]))]

In [91]:
#Spark para DataFrame
df_final_spark = ss.createDataFrame(df_time_final_rdd,['resultado','label','feature'])

In [92]:
df_final_spark.show(10)

+---------+-----+-----------------+
|resultado|label|          feature|
+---------+-----+-----------------+
|  vitoria|  2.0|[4.8,3.0,1.4,0.3]|
|  vitoria|  2.0|[5.1,3.8,1.6,0.2]|
|  vitoria|  2.0|[4.6,3.2,1.4,0.2]|
|  vitoria|  2.0|[5.3,3.7,1.5,0.2]|
|  vitoria|  2.0|[5.1,3.5,1.4,0.2]|
|  vitoria|  2.0|[4.9,3.0,1.4,0.2]|
|  vitoria|  2.0|[4.7,3.2,1.3,0.2]|
|  vitoria|  2.0|[4.6,3.1,1.5,0.2]|
|  vitoria|  2.0|[5.0,3.6,1.4,0.2]|
|  vitoria|  2.0|[5.4,3.9,1.7,0.4]|
+---------+-----+-----------------+
only showing top 10 rows



In [75]:
#Dados de treino e teste
(dados_treino,dados_teste) = df_final_spark.randomSplit([0.8,0.2])

### Machine Learning

In [93]:
dClassifier = DecisionTreeClassifier(maxDepth = 3,labelCol = 'label',featuresCol = 'features')

In [95]:
modelo = dClassifier.fit(dados_treino)

In [82]:
#Hiperparamentro padrao
modelo.numNodes

5

In [84]:
#Nosso hiperparametro
modelo.depth

2

In [85]:
previsoes = modelo.transform(dados_teste)

In [88]:
previsoes.select("resultado", "label", "prediction", "probability").collect()

[Row(resultado='derrota', label=0.0, prediction=0.0, probability=DenseVector([1.0, 0.0, 0.0])),
 Row(resultado='derrota', label=0.0, prediction=1.0, probability=DenseVector([0.0638, 0.9362, 0.0])),
 Row(resultado='derrota', label=0.0, prediction=0.0, probability=DenseVector([1.0, 0.0, 0.0])),
 Row(resultado='derrota', label=0.0, prediction=0.0, probability=DenseVector([1.0, 0.0, 0.0])),
 Row(resultado='vitoria', label=2.0, prediction=2.0, probability=DenseVector([0.0, 0.0, 1.0])),
 Row(resultado='vitoria', label=2.0, prediction=2.0, probability=DenseVector([0.0, 0.0, 1.0])),
 Row(resultado='vitoria', label=2.0, prediction=2.0, probability=DenseVector([0.0, 0.0, 1.0])),
 Row(resultado='vitoria', label=2.0, prediction=2.0, probability=DenseVector([0.0, 0.0, 1.0])),
 Row(resultado='vitoria', label=2.0, prediction=2.0, probability=DenseVector([0.0, 0.0, 1.0])),
 Row(resultado='vitoria', label=2.0, prediction=2.0, probability=DenseVector([0.0, 0.0, 1.0])),
 Row(resultado='vitoria', label=2.

In [89]:
avaliador = MulticlassClassificationEvaluator(predictionCol = 'prediction',labelCol = 'label',metricName = 'accuracy')

In [90]:
avaliador.evaluate(previsoes)

0.8518518518518519